# Pattern Matching

Since its seventh version, C# has been borrowing more and more functional concepts from its little brother, F#.

Among these concepts, pattern matching is probably the one that will affect most how code is written.

This notebook give a recap of all the patterns available up to C# 9.0 and how to use them.

## What is pattern matching

Before delving into the several patterns supported by the C# language, let’s introduce the concept of pattern matching.

In this specific context, pattern matching is the act of checking whether a given object matches certain criteria.

These criteria can range from “being an instance of a type” to “having a property whose value is within a range of values”.

## When to use pattern matchingPermalink

Pattern matching excels at traversing complex object structures when the type system cannot help us.

A good example is when exploring an object received from an external REST API.

Also, it can be used to create a finite-state machine.

## Where to use pattern matchingPermalink

Pattern matching is a check, therefore it can be used wherever we are introducing a branch in our code.

Typical scenarios are:

if statements, with the help of the keyword is,
switch statements and expressions.

## Test subject

The classes below will be our test subjects for this post.

You will notice two things.

Even if they clearly represent shapes, these classes share no common ancestor but for object.
Their properties use the new keyword init. This means that we can set the value only via the constructor or when initializing the object.

In [1]:
public class Square 
{
    public double Side { get; set; }
}

public class Circle
{
    public double Radius { get; set; }
}

public class Rectangle
{
    public double Length { get; set; }
    public double Height { get; set; }
}

public class Triangle
{
    public double Base { get; set; }
    public double Height { get; set; }
}

## Constant patterns

In [1]:
// Constant pattern is the most basice one, but rarely used by themselves.
var rect = new Rectangle{Height = 0, Length = 4};
if (rect.Height is 0)
{
    Console.WriteLine("The rect has no height");
}

In [1]:
// You can use == to achive the same, but be aware that == can be overrided
if (rect.Height == 0)
{
    Console.WriteLine("The rect has no height");
}

## Null pattern

In [1]:
// null pattern is a special case of constant pattern
if(rect is null)
{
    Console.WriteLine($"This rec is null");
}
else
{
    Console.WriteLine($"Rec: height:{rect.Height}, length:{rect.Length}");
}

In [1]:
// C# 9 come with logical pattern (explain later), and it make not null check more beautiful
// if(rect is not null)
// {
//     Console.WriteLine($"Rec: height:{rect.Height}, length:{rect.Length}");
// }

## Type pattern

In [1]:
// check a type and declare a variable of that type
var shape = new Square {Side = 5};
if (shape is Square sq)
{
    Console.WriteLine($"This shape is squre, its side is {sq.Side}");
}

In [1]:
// the above is equivalent to
if (shape is Square)
{
    var sq = shape as Square;
    Console.WriteLine($"This shape is squre, its side is {sq.Side}");
}

## Property pattern

In [1]:
// This pattern allow to explore the object's properties
var circle = new Circle{ Radius = 0};
if (circle is {Radius: 0})
{
    Console.WriteLine("Circle redius is 0");
}

In [1]:
// And it can be nested!
public class Person
{
    public string FirstName { get; set; }

    public string LastName { get; set; }

    public DateTime DateOfBirth { get; set; }

    public Person Father { get; set; }
    public Person Mother{ get; set; }
}

var person = new Person{ Father = new Person{DateOfBirth = new DateTime(1960, 1, 1)}};
if (person is { Father: { DateOfBirth: { Year: 1960 }}})
{
    Console.WriteLine("This person's father born in 1960");
}

## Relational pattern

In [1]:
// with relational pattern, comparision can be done
// circle.Radius = 5;
// if (circle is Circle {Radius: > 3})
// {
//     Console.WriteLine("this circle radius is over 3");
// }

The notation used for the relational patterns is equivalent to the comparison operators.

* \> for *greater than*
* \>= for *greater or equal than*
* < for *less than*
* <= for *less or equal than*

## Negated pattern

First stop, this pattern will make null check so much cleaner

In [1]:
// before, have to do this
if (shape != null)
{
    Console.WriteLine("shape is not null");
}

In [1]:
// now, just do this, it is like read plain english!
// if (shape is not null)
// {
//     Console.WriteLine("shape is null");
// }

In [1]:
// // Negated patterns can be prepended to any other pattern
// if (circle is Circle {Radius: not 0})
// {
//     Console.WriteLine("Radius is 0");
// }

## Conjunctive and Disjunctive pattern

Conjuntive and disjuntive patterns take their intimidating name from the logical operations known as conjunction and disjunction, and I just refer then as logical pattern.

Very much like the boolean operators && and || are used to pair expressions, the and and or keywords are used to pair patterns.

In [1]:
// // let's make comparision more pwoerful
// if (circle is Circle {Radius: > 0 and < 5} circle)
// {
//     Console.WriteLine("This is a cirlce with radius between 0 and 5");
// }

In [1]:
// now compaire to the old way of doing this
// isn't the newer way clear and consise?
if (circle is Circle)
{
    if (circle.Radius > 0 && circle.Radius < 100)
    {
        Console.WriteLine("This is a cirlce with radius between 0 and 5");
    }
}

Note the `and` and `or` pattern are not equivalent:

* `and` cannot be placed between two type patterns (unless they are targeting interfaces)
* `or` can be placed between two type patterns but it doesn’t support capturing
* `and` cannot be placed in a property pattern without a relational one
* `or` can be placed in a property pattern without a relational one and supports capturing
* `or` cannot be used between two properties of the same object
* `and` cannot be used between two properties of the same object, but it is implicit

Some example:

```csharp
shape is Square and Circle // this will not compile
shape is Square or Circle // OK!
shape is Square or Circle smt // this will not compile
shape is Square { Side: 0 and 1 } // this will not compile
shape is Square { Side: 0 or 1 } sq // OK!
shape is Rectangle { Height: 0 or Length: 0 } // this will not compile
shape is Rectangle { Height: 0 } or Rectangle { Length: 0 } // OK!
shape is Rectangle { Height: 0 and Length: 0 } // this will not compile
shape is Rectangle { Height: 0, Length: 0 } re // OK! equivalent to the pattern above
```

## Var pattern

While powerful and elegant, patterns can only be constant expressions.

`var` patterns help with many of the remaining cases by allowing us to declare a variable that is visible outside of the pattern being matched.

For example, let’s assure that the incoming shape is a Square whose side is a multiple of 2. Since patterns are constant expressions, we cannot perform the modulo calculation in the pattern.

In [1]:
// using var to use a property outside the scope
var squere = new Square{ Side =5 };
if (squere is Square { Side : var side} sq && side %2 == 1)
{
    Console.WriteLine("Side is odd");
}

## Tuple pattern

Some algorithms depend on multiple inputs. Tuple patterns allow you to switch based on multiple values expressed as a tuple.

In [1]:
// let's write a FizzBuzz with ease
public void FizzBuzz(Int32 limit)
{
    Enumerable.Range(1, limit)
        .Select(i => (i % 3 ==0, i % 5 == 0, i) switch
        {
            (true, false, _) => "Fizz",
            (false, true, _) => "Buzz",
            (true, true, _) => "FizzBuzz",
            (_, _, var n) => $"{n}"
        })
        .ToList()
        .ForEach(s => Console.WriteLine(s));
}

In [1]:
FizzBuzz(10)

## Positional pattern

Positional patterns come handy when working with types that are decorated with a deconstructor. In such cases, it is possible to define a pattern based on the values the object is deconstructed into.

In [1]:
public struct Rectangle
{
    public double Length { get; set; }
    public double Height { get; set; }

    public void Deconstruct(out double height, out double length)
    {
        height = Height;
        length = Length;
    }
}

In [1]:
// with that in hand, we can create patterns somehow similar to the tuple ones.
var rect = new Rectangle{Height = 10, Length = 5};
if (rect is (10, _) re)
{
    Console.WriteLine($"Rec height is {rect.Length}");
}